In [1]:
from ultralytics import YOLO
import cv2
import time

# Load the model
model = YOLO('shakil_l.pt')

# Open a connection to the camera
cap = cv2.VideoCapture(1)  # Change the argument to 1 or another number if you have multiple cameras

if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

# Dictionary mapping class names to colors (BGR format)
colors = {
    "Safety signs/ Text signs": (0, 255, 255), 
    "Lab Tables": (255, 0, 0), 
    "Lab Chair(wooden)": (19, 69, 139), 
    "Lab chair(stool)": (139, 0, 0), 
    "3D Printer": (0, 165, 255), 
    "Oscilloscopes": (255, 255, 0), 
    "Signal Generators": (255, 0, 255), 
    "Power Supply": (128, 0, 128), 
    "TV": (0, 0, 139), 
    "PC": (173, 216, 230), 
    "Robot Manipulator": (255, 192, 203), 
    "Whiteboard": (255, 255, 255), 
    "Factory Automation plant": (0, 128, 128), 
    "Equipment storage cupboards": (169, 169, 169), 
    "Laptops": (211, 211, 211), 
    "A/C": (224, 255, 255), 
    "Probes": (144, 238, 144), 
    "Table Plug Points": (0, 0, 128), 
    "Lecture Table": (128, 0, 0), 
    "Lecture Chair 1": (0, 100, 0), 
    "Lecture Chair 2": (128, 128, 0), 
    "Projector": (0, 140, 255), 
    "Stage": (42, 42, 128), 
    "Podium": (139, 69, 19), 
    "Smart board": (224, 255, 255), 
    "Speakers": (0, 215, 255), 
    "Pitch": (0, 128, 0), 
    "Stumps": (245, 245, 220), 
    "Goal posts": (192, 192, 192), 
    "Bats": (0, 100, 255), 
    "Ball": (203, 192, 255), 
    "Trees": (0, 255, 0), 
    "Road": (128, 128, 128), 
    "Buildings": (169, 169, 169), 
    "Vehicle": (0, 0, 255), 
    "Light post": (224, 255, 255), 
    "Tele-Post": (19, 69, 139), 
    "Gate": (139, 0, 0), 
    "Fence": (0, 128, 0), 
    "Security Checkpoint": (169, 169, 169), 
    "Lab-table02": (107, 142, 35)
}

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    # Perform detection
    result = model([frame])

    # Initialize list to store detected boxes and labels
    detect = []

    # Iterate over detected boxes
    for box in result[0].boxes:
        if box.conf[0].item() > 0.40:
            cords = box.xyxy[0].tolist()
            cords = [round(x) for x in cords]
            x1, y1, x2, y2 = cords

            # Get the label
            label1 = box.cls[0].item()  # `box.cls` contains the class index

            # Convert class index to class name if available
            class_names = [
                "Safety signs", 
                "Lab Tables", 
                "Lab Chair(wooden)", 
                "Lab chair(stool)", 
                "3D Printer", 
                "Oscilloscopes", 
                "Signal Generators", 
                "Power Supply", 
                "TV", 
                "PC", 
                "Robot Manipulator", 
                "Whiteboard", 
                "Factory Automation plant", 
                "Equipment storage cupboards", 
                "Laptops", 
                "A/C", 
                "Probes", 
                "Table Plug Points", 
                "Lecture Table", 
                "Lecture Chair 1", 
                "Lecture Chair 2", 
                "Projector", 
                "Stage", 
                "Podium", 
                "Smart board", 
                "Speakers", 
                "Pitch", 
                "Stumps", 
                "Goal posts", 
                "Bats", 
                "Ball", 
                "Trees", 
                "Road", 
                "Buildings", 
                "Vehicle", 
                "Light post", 
                "Tele-Post", 
                "Gate", 
                "Fence", 
                "Security Checkpoint", 
                "Lab-table02"
            ]  # Class names
            class_name = class_names[int(label1)]

            # Add the detected box and label to the list
            detect.append((x1, y1, x2, y2, class_name))

    # Draw bounding boxes with labels and semi-transparent highlights
    for (x1, y1, x2, y2, class_name) in detect:
        # Get the color for the current class
        color = colors.get(class_name, (0, 0, 255))  # Default to red if class is not found

        # Create a semi-transparent overlay
        overlay = frame.copy()
        alpha = 0.3  # Transparency factor

        # Draw the semi-transparent rectangle
        cv2.rectangle(overlay, (x1, y1), (x2, y2), color=color, thickness=-1)
        cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

        # Determine if the text should be black or white based on the brightness of the color
        brightness = int(color[0] * 0.299 + color[1] * 0.587 + color[2] * 0.114)
        text_color = (0, 0, 0) if brightness > 128 else (255, 255, 255)  # Use black for light colors, white for dark

        # Draw the bounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), color=color, thickness=2)

        # Create the label text
        label_text = f"{class_name}"

        # Choose a smaller font and calculate the label size
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.4
        font_thickness = 1
        label_size = cv2.getTextSize(label_text, font, font_scale, font_thickness)[0]

        # Set the label background
        label_background = (x1, y1 - label_size[1] - 10, x1 + label_size[0], y1)
        cv2.rectangle(frame, (label_background[0], label_background[1]), (label_background[2], label_background[3]), color, -1)

        # Put the label text above the bounding box
        cv2.putText(frame, label_text, (x1, y1 - 5), font, font_scale, text_color, font_thickness)

    # Display the resulting frame
    cv2.imshow('Real-time Detection', frame)

    # Wait for 1 second
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    time.sleep(0.01)

# When everything is done, release the capture and close the window
cap.release()
cv2.destroyAllWindows()



0: 480x640 1 Laptop, 1528.0ms
Speed: 4.9ms preprocess, 1528.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Safety_Sign, 1 Lab_Chair_02, 1 Whiteboard, 2 Plugpointss, 1517.0ms
Speed: 5.0ms preprocess, 1517.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Safety_Sign, 1 Lab_Chair_02, 1 Whiteboard, 2 Plugpointss, 1566.0ms
Speed: 4.0ms preprocess, 1566.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Safety_Sign, 1 Lab_Chair_02, 1 Whiteboard, 2 Plugpointss, 1537.0ms
Speed: 3.0ms preprocess, 1537.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Safety_Sign, 1 Lab_Chair_02, 1 Whiteboard, 2 Plugpointss, 1566.0ms
Speed: 3.0ms preprocess, 1566.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Safety_Sign, 1 Lab_Chair_02, 1 Whiteboard, 2 Plugpointss, 1583.0ms
Speed: 3.0ms preprocess, 1583.0ms inference, 3.0ms postprocess per im